In [1]:
from alphatools.research import run_pipeline, make_factor_plot
from alphatools.expression import ExpressionAlpha
from alphatools.ics import Sector
from zipline.pipeline.factors import AverageDollarVolume
from zipline.pipeline import Pipeline

In [2]:
universe = AverageDollarVolume(window_length=120).top(500)

In [3]:
expressions = {
    8:  '-1*rank(((sum(opens, 5)*sum(returns, 5))-delay((sum(opens, 5)*sum(returns, 5)),10)))', 
    53: '(-1*delta((((close-low) - (high-close)) / (close-low)), 9))',
    101: '((close - opens) / ((high - low) + .001))',
    102: 'scale((close > delay(close, 1)) ? opens : delay(opens, 1))',
    103: '-rank(stddev(returns, 60))*log(close/delay(close,5))',
    104: 'product(1+returns, 20)',
    105: 'decay_linear((close/delay(close,5))-1, 5)',
    'test': 'decay_linear(close,5)'
}


In [4]:
e = ExpressionAlpha(expressions['test'])

In [5]:
e.parse()
print(e.tree.pretty())

decay_linear
  close
  5



In [6]:
e.transform()

<alphatools.expression.expression.ExpressionAlpha instance at 0x1c1eba42d8>

In [7]:
e.generate_pipeline_code()
print(e.pipeline_code)

class ExprAlpha_1(CustomFactor):
    inputs = [Returns(window_length=2), USEP.open, USEP.high,
              USEP.low, USEP.close, USEP.volume]
    window_length = 7

    def compute(self, today, assets, out, returns, opens, high, low, close, volume):
        v0 = (np.arange(5)+1.)/np.sum(np.arange(5)+1.)
        v1 = pd.DataFrame(data=close).rolling(
            window=5, center=False, min_periods=5).apply(lambda x: (x*v0).sum()).values
        out[:] = v1[-1]



In [8]:
expr_factor = (
    ExpressionAlpha('rank(log(close/delay(close, 5)))').
    make_pipeline_factor().
    pipeline_factor(mask=universe)
)

In [9]:
from __future__ import division

In [10]:
p = Pipeline(screen=universe)
p.add(e.make_pipeline_factor().pipeline_factor(mask=universe), "Expression Alpha")

start_date = '2017-01-06'
end_date = '2017-01-06'

df = run_pipeline(p, start_date, end_date)

In [11]:
df

Expression Alpha
2017-01-06 00:00:00+00:00 Equity(0 [A])               46.488667
                          Equity(2 [AAL])             46.385333
                          Equity(7 [AAP])            171.181333
                          Equity(8 [AAPL])           116.263333
                          Equity(12 [ABBV])           63.147333
                          Equity(13 [ABC])            82.706000
                          Equity(20 [ABT])            39.214667
                          Equity(37 [ACN])           116.170667
                          Equity(45 [ADBE])          104.408667
                          Equity(48 [ADI])            72.158333
                          Equity(49 [ADM])            45.901333
                          Equity(52 [ADP])           103.290000
                          Equity(53 [ADS])           234.366667
                          Equity(54 [ADSK])           76.418000
                          Equity(66 [AEO])            15.370000
                          Equity(67 [AEP])            62.988667
                          Equity(71 [AET])           123.964000
                          Equity(77 [AFL])            69.982000
                          Equity(85 [AGN])           216.146000
                          Equity(98 [AIG])            65.557333
                          Equity(109 [AKAM])          68.278667
                          Equity(114 [AKS])           10.745333
                          Equity(116 [ALB])           89.208667
                          Equity(126 [ALK])           88.367333
                          Equity(128 [ALL])           74.177333
                          Equity(136 [ALXN])         129.370667
                          Equity(138 [AMAT])          32.165267
                          Equity(145 [AMD])           11.365333
                          Equity(146 [AME])           49.244000
                          Equity(149 [AMGN])         151.251333
...                                                         ...
                          Equity(3067 [WDC])          69.835333
                          Equity(3070 [WEC])          58.832000
                          Equity(3077 [WFC])          55.544000
                          Equity(3079 [WFM])          30.750667
                          Equity(3085 [WHR])         184.473333
                          Equity(3095 [WLL])          12.763333
                          Equity(3098 [WLTW])        124.352667
                          Equity(3099 [WM])           70.574667
                          Equity(3101 [WMB])          31.454000
                          Equity(3105 [WMT])          69.051333
                          Equity(3107 [WNR])          37.478000
                          Equity(3111 [WPX])          14.951333
                          Equity(3117 [WRK])          51.728000
                          Equity(3122 [WSM])          49.395333
                          Equity(3137 [WWAV])         55.520667
                          Equity(3142 [WY])           30.592000
                          Equity(3144 [WYNN])         89.392667
                          Equity(3145 [X])            35.649333
                          Equity(3148 [XEC])         138.602667
                          Equity(3149 [XEL])          40.760000
                          Equity(3151 [XLNX])         58.860000
                          Equity(3156 [XOM])          89.723333
                          Equity(3163 [XRAY])         58.508000
                          Equity(3165 [XRX])           7.403333
                          Equity(3171 [YELP])         39.025333
                          Equity(3172 [YHOO])         39.974667
                          Equity(3175 [YUM])          63.462000
                          Equity(3180 [ZBH])         104.117333
                          Equity(3188 [ZION])         43.270667
                          Equity(3197 [ZTS])          53.836000

[500 rows x 1 columns]

In [11]:
make_factor_plot(df, start_date, end_date);

Formatting factor data for: Expression Alpha
Dropped 100.0% entries from factor data: 1.3% in forward returns computation and 98.7% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).


MaxLossExceededError: max_loss (35.0%) exceeded 100.0%, consider increasing it.

In [27]:
e = ExpressionAlpha('rank(log(close/delay(opens, 5)))').parse().transform().generate_pipeline_code()

In [28]:
print(e.pipeline_code)

class ExprAlpha_1(CustomFactor):
    inputs = [Returns(window_length=2), USEP.open, USEP.high,
              USEP.low, USEP.close, USEP.volume]
    window_length = 7

    def compute(self, today, assets, out, returns, opens, high, low, close, volume):
        v22 = np.roll(opens, 5, axis=0)
        v23 = close / v22
        v24 = np.log(v23)
        v25 = np.apply_along_axis(rankdata, 1, v24)
        out[:] = v25[-1]



In [29]:
a

NameError: name 'a' is not defined

In [18]:
b

array([[ 0.54254437,  0.14217005],
       [ 0.37334076,  0.67413362],
       [ 0.44183317,  0.43401399],
       [ 0.61776698,  0.51313824],
       [ 0.65039718,  0.60103895],
       [ 0.8052232 ,  0.52164715],
       [ 0.90864888,  0.31923609],
       [ 0.09045935,  0.30070006],
       [ 0.11398436,  0.82868133],
       [ 0.04689632,  0.62628715]])

In [86]:
def column_corr((a, b), window=-1):
    if window==-1:
        window = a.shape[0]
    out = np.zeros(shape = (1, a.shape[1]))
    for i in xrange(a.shape[1]-1):
        out[i] = pearsonr(a[-window:, i], b[-window:, i])[0]
    return out
        

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
np.random.seed(10)
a = np.random.random(30).reshape(10,3)
b = np.random.random(30).reshape(10,3)

In [2]:
a_1 = pd.DataFrame(a)
b_1 = pd.DataFrame(b)



In [15]:
def roll_corr((a, b), window):
    out = np.ones_like(a)*np.nan
    for i in xrange(window-1, a.shape[0]):   # 2- 10
        #print "%d --> %d" % ((i-(window-1)), i)
        for j in xrange(a.shape[1]):
            out[i, j] = pearsonr(
                a[(i-(window-1)):(i), j], b[(i-(window-1)):(i), j]
            )[0]
    return out

In [16]:
%%timeit
out = roll_corr((a,b), window=5)

1000 loops, best of 3: 645 µs per loop


In [23]:
%%timeit
out = pd.rolling_corr(arg1=a_1, arg2=b_1, window=5)

/anaconda3/envs/py27_clean/lib/python2.7/site-packages/ipykernel_launcher.py:257: FutureWarning: pd.rolling_corr is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=5).corr(other=<DataFrame>)


100 loops, best of 3: 10.1 ms per loop


In [12]:
out[-1]

NameError: name 'out' is not defined

In [5]:
#from __future__ import division
v0 = (np.arange(5)+1.)/np.sum(np.arange(5)+1.)

out = a_1.rolling(5).apply(lambda x: (x*v0).sum()).values

In [6]:
v0

array([ 0.06666667,  0.13333333,  0.2       ,  0.26666667,  0.33333333])

In [27]:
out[-1, :]

array([ 0.48050722,  0.55059807,  0.59170856])

In [18]:
from numba import jit

In [19]:
@jit
def roll_corr2((a, b), window):
    out = np.ones_like(a)*np.nan
    for i in xrange(window-1, a.shape[0]):   # 2- 10
        #print "%d --> %d" % ((i-(window-1)), i)
        for j in xrange(a.shape[1]):
            out[i, j] = pearsonr(
                a[(i-(window-1)):(i), j], b[(i-(window-1)):(i), j]
            )[0]
    return out

ValueError: '.0' is not a valid parameter name

In [207]:
def scale(x):
    return (x - np.nanmean(x))/np.nansum(np.abs(x - np.nanmean(x)))

In [211]:
np.apply_along_axis(lambda x: (x - np.nanmean(x))/np.nansum(np.abs(x - np.nanmean(x))), 1, a)

array([[ 0.32572934, -0.5       ,  0.17427066],
       [ 0.4853247 ,  0.0146753 , -0.5       ],
       [-0.23118186,  0.5       , -0.26881814],
       [-0.5       ,  0.11250682,  0.38749318],
       [-0.5       ,  0.07890222,  0.42109778],
       [ 0.14084777,  0.35915223, -0.5       ],
       [ 0.5       , -0.02694105, -0.47305895],
       [-0.45819778, -0.04180222,  0.5       ],
       [-0.23369585, -0.26630415,  0.5       ],
       [-0.5       ,  0.41441077,  0.08558923]])

array([[ 0.77132064,  0.02075195,  0.63364823],
       [ 0.74880388,  0.49850701,  0.22479665],
       [ 0.19806286,  0.76053071,  0.16911084],
       [ 0.08833981,  0.68535982,  0.95339335],
       [ 0.00394827,  0.51219226,  0.81262096],
       [ 0.61252607,  0.72175532,  0.29187607],
       [ 0.91777412,  0.71457578,  0.54254437],
       [ 0.14217005,  0.37334076,  0.67413362],
       [ 0.44183317,  0.43401399,  0.61776698],
       [ 0.51313824,  0.65039718,  0.60103895]])

In [33]:
from __future__ import division

np.sum()

1.0

55

In [196]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
np.random.seed(2)
a = np.random.random(30).reshape(10,3)    # days x stocks
b = np.random.randint(2, 5, size=(1,3))   # Industry cat vars as integers

In [197]:
a

array([[ 0.4359949 ,  0.02592623,  0.54966248],
       [ 0.43532239,  0.4203678 ,  0.33033482],
       [ 0.20464863,  0.61927097,  0.29965467],
       [ 0.26682728,  0.62113383,  0.52914209],
       [ 0.13457995,  0.51357812,  0.18443987],
       [ 0.78533515,  0.85397529,  0.49423684],
       [ 0.84656149,  0.07964548,  0.50524609],
       [ 0.0652865 ,  0.42812233,  0.09653092],
       [ 0.12715997,  0.59674531,  0.226012  ],
       [ 0.10694568,  0.22030621,  0.34982629]])

In [198]:
b

array([[4, 2, 4]])

In [199]:
ohe = np.zeros((b.size, b.max()+1))
ohe[np.arange(b.size),b] = 1

In [191]:
ohe.T

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  0.],
       [ 1.,  0.,  1.]])

In [200]:
ind_mean = np.nan_to_num(a.dot(ohe)/ohe.sum(axis=0))

In [203]:
ind_mean

array([[ 0.        ,  0.        ,  0.02592623,  0.        ,  0.49282869],
       [ 0.        ,  0.        ,  0.4203678 ,  0.        ,  0.38282861],
       [ 0.        ,  0.        ,  0.61927097,  0.        ,  0.25215165],
       [ 0.        ,  0.        ,  0.62113383,  0.        ,  0.39798468],
       [ 0.        ,  0.        ,  0.51357812,  0.        ,  0.15950991],
       [ 0.        ,  0.        ,  0.85397529,  0.        ,  0.63978599],
       [ 0.        ,  0.        ,  0.07964548,  0.        ,  0.67590379],
       [ 0.        ,  0.        ,  0.42812233,  0.        ,  0.08090871],
       [ 0.        ,  0.        ,  0.59674531,  0.        ,  0.17658599],
       [ 0.        ,  0.        ,  0.22030621,  0.        ,  0.22838598]])

In [206]:
per_stock_ind_mean = ind_mean.dot(ohe.T)

In [207]:
per_stock_ind_mean # per stock ind_mean by day!

array([[ 0.49282869,  0.02592623,  0.49282869],
       [ 0.38282861,  0.4203678 ,  0.38282861],
       [ 0.25215165,  0.61927097,  0.25215165],
       [ 0.39798468,  0.62113383,  0.39798468],
       [ 0.15950991,  0.51357812,  0.15950991],
       [ 0.63978599,  0.85397529,  0.63978599],
       [ 0.67590379,  0.07964548,  0.67590379],
       [ 0.08090871,  0.42812233,  0.08090871],
       [ 0.17658599,  0.59674531,  0.17658599],
       [ 0.22838598,  0.22030621,  0.22838598]])